In [1]:
%%capture
pip install ibm_db

In [2]:
%%capture
pip install spacy

In [3]:
%%capture
pip install tabulate

In [4]:
import pandas as pd
import numpy as np
import re
import nltk 
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import ibm_db
import ibm_db_dbi
import string
from ast import literal_eval
from tabulate import tabulate
import spacy

# Database Connection

In [5]:
#Creating variable for DB connection
dsn_hostname = "54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud"
dsn_uid = "qgs98802"  
dsn_pwd = "f20uBnloVxeaSeZK"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "bludb"

dsn_port = "32733"
dsn_protocol = "TCPIP"  
 
dsn_security = "SSL"

In [6]:
# Formatting dsn in order to make connection
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd, dsn_security)

In [7]:
#Connecting to Database
try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

Connected to database:  bludb as user:  qgs98802 on host:  54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud


In [8]:
pconn = ibm_db_dbi.Connection(conn)

# Data

In [9]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [10]:
twitter_query = "SELECT CONTENT, COUNTRY FROM twitter_data"
twitter_data = pd.read_sql(twitter_query, pconn)
twitter_data['CONTENT'] = [remove_emoji(content[2:-2]) for content in twitter_data['CONTENT']]

reddit_query = "SELECT CONTENT, COUNTRY from reddit_data"
reddit_data = pd.read_sql(reddit_query, pconn)
reddit_data['CONTENT'] = [remove_emoji(content) for content in reddit_data['CONTENT']]

youtube_query = "SELECT CONTENT, COUNTRY from youtube_data"
youtube_data = pd.read_sql(youtube_query, pconn)
youtube_data['CONTENT'] = [remove_emoji(content) for content in youtube_data['CONTENT']]

In [11]:
total_data = pd.concat([twitter_data, reddit_data, youtube_data], axis=0, ignore_index=True)

# Concordancer

In [12]:
stopwords = nltk.corpus.stopwords.words('spanish')

In [13]:
%%capture
pip install https://github.com/explosion/spacy-models/releases/download/es_core_news_md-3.2.0/es_core_news_md-3.2.0.tar.gz

In [14]:
nlp = spacy.load("es_core_news_md")

In [15]:
def tokenizer(df):
    content_list = [content for content in df["CONTENT"]]
    tokenized_list = [re.findall(r'\w+', content) for content in content_list]
    return tokenized_list
    

In [16]:
def concordance(search_term, tokens, df, data_option=0):
    tokenized_data = tokenizer(df)
    concordance_data = []
    left_words = []
    right_words = []
    for post in range (len(tokenized_data)):
        for index in range(len(tokenized_data[post])):
            if tokenized_data[post][index].lower() == search_term:
                l = ' '.join(tokenized_data[post][index-tokens:index])
                r = ' '.join(tokenized_data[post][index+1: index+tokens+1])
                
                concordance_data.append([l, search_term, r])
                left_words.append(l)
                right_words.append(r)
    if data_option == 0:
        print(tabulate(concordance_data))
    elif data_option == 1:
        return [left_words, right_words]
    elif data_option == 2:
        return len(concordance_data)

In [17]:
def lemmatize(list_):
    lemma_list = []
    string_of_words=' '.join(list_)
    tokens = nlp(string_of_words)
    for token in tokens:
        lemma = token.lemma_
        lemma_list.append(lemma)
    return lemma_list

In [18]:
def clean_stop_words(list_):
    cleaned_words = [word for word in list_ if word.lower() not in stopwords and len(word) > 2]
    return cleaned_words

In [19]:
def comparative_concordance(df_list, search_term, country_list):
    term_dictionary = {}
    n = 0
    for dataframe in df_list:
        phrase_lists = concordance(search_term, 4, dataframe, data_option=1)
        frequency = concordance(search_term, 4, dataframe, data_option=2)
        phrase_list = phrase_lists[0] + phrase_lists[1]
        word_list = []
        for entry in phrase_list:
            words = entry.split(' ')
            word_list.extend(words)
        #get rid of stop words
        word_list_no_stop = clean_stop_words(word_list)
        #lemmatize all words
        word_list_lemmatized = lemmatize(word_list_no_stop)
        #standardizing capitalization
        word_list_cleaned = [word.lower() for word in word_list_lemmatized]
        counted_word_list = Counter(word_list_cleaned).most_common(10)
        term_dictionary[f'{country_list[n]}:{frequency}'] = counted_word_list
        n+=1
    return pd.DataFrame.from_dict(term_dictionary)
    

# Concordance of Chosen Terms

In [20]:
terms = ['derecha', 'izquierda', 'comunismo', 'socialismo', 'economia','gobierno', 'constitución', 'democracia',
        'libertad', 'partido', 'presidente']
country_list = ['Mexico', 'Colombia', 'Ecuador', 'Spain', 'Argentina', 'Chile', 'Peru', 'Total']

In [21]:
mexico_df = total_data[total_data['COUNTRY']== 'Mexico']
colombia_df = total_data[total_data['COUNTRY']=='Colombia']
ecuador_df = total_data[total_data['COUNTRY']=='Ecuador']
spain_df = total_data[total_data['COUNTRY']=='Spain']
argentina_df = total_data[total_data['COUNTRY']=='Argentina']
chile_df = total_data[total_data['COUNTRY']== 'Chile']
peru_df = total_data[total_data['COUNTRY']=='Peru']
country_df = [mexico_df, colombia_df, ecuador_df, spain_df, argentina_df, chile_df, peru_df, total_data]

### "Derecha"

In [22]:
comparative_concordance(country_df, terms[0], country_list)

,Mexico:160,Colombia:300,Ecuador:162,Spain:387,Argentina:146,Chile:684,Peru:360,Total:2199
0,"(izquierdo, 23)","(colombiano, 53)","(ecuatoriano, 31)","(español, 100)","(izquierdo, 16)","(chileno, 101)","(izquierdo, 71)","(izquierdo, 263)"
1,"(izquierda, 19)","(político, 41)","(político, 27)","(político, 75)","(político, 11)","(izquierdo, 64)","(peruano, 71)","(político, 260)"
2,"(mexicano, 19)","(centro, 40)","(izquierdo, 26)","(extremo, 57)","(argentina, 10)","(político, 60)","(extremo, 34)","(extremo, 153)"
3,"(político, 12)","(izquierdo, 31)","(ecuador, 14)","(izquierdo, 32)","(mano, 7)","(extremo, 42)","(político, 34)","(izquierda, 130)"
4,"(líder, 9)","(izquierda, 20)","(izquierda, 11)","(españa, 30)","(conservador, 7)","(chile, 37)","(izquierda, 34)","(centro, 107)"
5,"(mexicana, 9)","(colombia, 18)","(política, 8)","(mediático, 19)","(argentino, 6)","(centro, 29)","(peruana, 16)","(chileno, 103)"
6,"(repasar, 8)","(política, 13)","(extremo, 7)","(izquierda, 16)","(ver, 6)","(candidato, 27)","(perú, 13)","(español, 102)"
7,"(política, 7)","(ser, 12)","(centro, 7)","(judicial, 16)","(económico, 6)","(izquierda, 26)","(partido, 12)","(peruano, 71)"
8,"(ser, 7)","(ahora, 8)","(poder, 6)","(partido, 13)","(gobierno, 5)","(chilena, 22)","(centro, 11)","(política, 63)"
9,"(intenso, 7)","(extremo, 8)","(ecuatoriana, 6)","(policial, 13)","(milei, 5)","(hacer, 20)","(ser, 11)","(ser, 61)"


### "Izquierda

In [23]:
comparative_concordance(country_df, terms[1], country_list)

,Mexico:216,Colombia:255,Ecuador:126,Spain:335,Argentina:221,Chile:622,Peru:420,Total:2195
0,"(derecho, 34)","(derecho, 37)","(derecho, 21)","(español, 60)","(social, 16)","(político, 69)","(peruano, 73)","(derecho, 281)"
1,"(político, 21)","(político, 36)","(político, 14)","(político, 45)","(argentina, 12)","(derecho, 66)","(derecho, 71)","(político, 227)"
2,"(mexicano, 21)","(colombiano, 30)","(derecha, 14)","(derecho, 40)","(derecho, 11)","(extremo, 65)","(político, 35)","(extremo, 122)"
3,"(centro, 12)","(centro, 27)","(ecuador, 13)","(extremo, 22)","(frente, 10)","(chileno, 60)","(derecha, 30)","(centro, 112)"
4,"(ser, 11)","(colombia, 22)","(ecuatoriano, 10)","(españa, 18)","(derecha, 8)","(centro, 42)","(centro, 21)","(derecha, 110)"
5,"(méxico, 11)","(derecha, 16)","(democrática, 10)","(https, 16)","(diario, 8)","(chile, 36)","(ser, 19)","(ser, 78)"
6,"(derecha, 9)","(extremo, 11)","(partido, 7)","(política, 13)","(político, 7)","(derecha, 26)","(radical, 17)","(peruano, 74)"
7,"(política, 9)","(izquierdo, 8)","(izquierda, 7)","(poder, 10)","(ser, 7)","(ser, 26)","(extremo, 16)","(español, 62)"
8,"(progresista, 7)","(solo, 8)","(gobierno, 6)","(siempre, 10)","(argentino, 7)","(boric, 23)","(mismo, 14)","(https, 61)"
9,"(izquierdo, 7)","(colombiana, 8)","(política, 4)","(psoe, 8)","(hacer, 6)","(candidato, 23)","(solo, 12)","(chileno, 60)"


### "Comunismo"

In [24]:
comparative_concordance(country_df, terms[2], country_list)

,Mexico:20,Colombia:36,Ecuador:5,Spain:48,Argentina:23,Chile:153,Peru:67,Total:352
0,"(socialismo, 4)","(colombia, 10)","(hablar, 2)","(libertad, 5)","(intervencionista, 3)","(chile, 36)","(socialismo, 6)","(chile, 36)"
1,"(comunismo, 2)","(socialismo, 3)","(socialismo, 2)","(socialismo, 4)","(vender, 2)","(boric, 7)","(perú, 4)","(socialismo, 25)"
2,"(avanzar, 2)","(https, 3)","(correa, 2)","(social, 4)","(ilusión, 2)","(https, 7)","(país, 4)","(https, 17)"
3,"(nel, 2)","(ojoconel22, 2)","(esencia, 1)","(pro, 4)","(hablar, 2)","(chileno, 6)","(https, 4)","(político, 12)"
4,"(idea, 2)","(uribismo, 2)","(robo, 1)","(terruco, 4)","(comunismo, 2)","(socialismo, 6)","(político, 3)","(decir, 11)"
5,"(méxico, 2)","(socialimo, 2)","(mañana, 1)","(complices, 4)","(mas, 2)","(venezuela, 6)","(mismo, 3)","(libertad, 10)"
6,"(toca, 2)","(decir, 2)","(cualquiera, 1)","(alejar, 3)","(miseria, 2)","(política, 5)","(querer, 3)","(colombia, 10)"
7,"(conciencia, 1)","(idea, 2)","(peor, 1)","(decir, 3)","(aparece, 2)","(vender, 5)","(castillo, 3)","(venezuela, 9)"
8,"(querer, 1)","(lado, 2)","(etc, 1)","(españa, 3)","(quot, 2)","(ilusión, 5)","(llevar, 3)","(querer, 8)"
9,"(instaurar, 1)","(comunista, 2)","(tendencia, 1)","(político, 2)","(https, 2)","(salvar, 5)","(peruano, 2)","(país, 8)"


### "Socialismo"

In [25]:
comparative_concordance(country_df, terms[3], country_list)

,Mexico:34,Colombia:26,Ecuador:21,Spain:21,Argentina:46,Chile:22,Peru:24,Total:194
0,"(capitalismo, 4)","(xxi, 3)","(siglo, 5)","(comunismo, 4)","(argentina, 3)","(comunismo, 6)","(comunismo, 6)","(comunismo, 23)"
1,"(comunismo, 3)","(comunismo, 2)","(xxi, 3)","(pacto, 2)","(siempre, 3)","(siglo, 3)","(entonces, 2)","(siglo, 11)"
2,"(trabajar, 3)","(colombia, 2)","(hablar, 2)","(nacional, 2)","(funcionar, 3)","(xxi, 3)","(libre, 2)","(xxi, 10)"
3,"(oposición, 2)","(contención, 2)","(comunismo, 2)","(diferencia, 2)","(querer, 2)","(chile, 3)","(politicaperuana, 2)","(político, 8)"
4,"(saber, 2)","(lado, 2)","(corrupción, 2)","(igual, 2)","(kirchnerismo, 2)","(político, 2)","(rumi, 1)","(nel, 7)"
5,"(fracaso, 2)","(socialismo, 2)","(ecuador, 2)","(europeo, 2)","(político, 2)","(hacer, 2)","(revistarumi, 1)","(capitalismo, 7)"
6,"(ser, 2)","(nel, 2)","(xxl, 2)","(político, 2)","(nel, 2)","(mezclado, 2)","(cultura, 1)","(siempre, 6)"
7,"(equivalente, 2)","(cómo, 2)","(fracasado, 2)","(extranjero, 1)","(mas, 2)","(boric, 2)","(revistapolitica, 1)","(querer, 5)"
8,"(enserio, 2)","(politica, 2)","(hijodelsuelo, 1)","(nel, 1)","(junto, 2)","(mismo, 2)","(elevar, 1)","(decir, 5)"
9,"(creser, 2)","(siglo, 2)","(jorecuador, 1)","(pilar, 1)","(votar, 2)","(progresismo, 2)","(louis, 1)","(ser, 5)"


### "Economia"

In [26]:
comparative_concordance(country_df, terms[4], country_list)

,Mexico:35,Colombia:11,Ecuador:11,Spain:17,Argentina:143,Chile:15,Peru:14,Total:246
0,"(mexicano, 7)","(colombia, 7)","(ecuador, 5)","(psoe, 7)","(com, 60)","(ayuda, 3)","(peru, 3)","(com, 65)"
1,"(moodys, 5)","(politico, 5)","(politico, 4)","(sánchez, 6)","(www, 54)","(etc, 2)","(peruano, 3)","(www, 60)"
2,"(asegurar, 5)","(juventud, 5)","(radioaguila, 3)","(politica, 6)","(https, 52)","(ver, 2)","(democracia, 2)","(https, 60)"
3,"(politico, 4)","(igualdad, 5)","(entrevista, 3)","(crisis, 6)","(politico, 33)","(mayor, 1)","(influir, 2)","(politico, 49)"
4,"(gobierno, 4)","(mejorescondición, 5)","(radio, 3)","(poder, 6)","(cronista, 32)","(avance, 1)","(fiscalia, 2)","(politica, 45)"
5,"(politica, 3)","(politica, 2)","(politica, 2)","(vox_es, 6)","(politica, 31)","(ife, 1)","(ministerio, 2)","(cronista, 32)"
6,"(nhttps, 3)","(adelante, 2)","(pensar, 2)","(https, 4)","(argentina, 25)","(matar, 1)","(vivienda, 2)","(argentina, 26)"
7,"(pais, 3)","(https, 2)","(comparte, 2)","(ministros, 4)","(fmi, 11)","(todo, 1)","(nhttps, 2)","(fmi, 11)"
8,"(cadenapolitica, 2)","(www, 2)","(https, 2)","(político, 3)","(lanacion, 9)","(isso, 1)","(hablar, 1)","(nhttps, 11)"
9,"(cadenaradio, 2)","(infobae, 2)","(creo, 1)","(españa, 3)","(nhttps, 6)","(precisar, 1)","(base, 1)","(gobierno, 10)"


### "Gobierno"

In [27]:
comparative_concordance(country_df, terms[5], country_list)

,Mexico:665,Colombia:368,Ecuador:482,Spain:543,Argentina:610,Chile:306,Peru:455,Total:3429
0,"(méxico, 116)","(político, 33)","(político, 113)","(españa, 110)","(hacer, 36)","(chile, 21)","(castillo, 35)","(político, 286)"
1,"(política, 55)","(colombia, 29)","(económico, 61)","(político, 43)","(nacional, 23)","(piñera, 21)","(plan, 23)","(política, 144)"
2,"(biden, 53)","(colombiano, 24)","(ecuador, 60)","(hacer, 24)","(macri, 21)","(programa, 18)","(palacio, 22)","(https, 136)"
3,"(político, 41)","(duque, 20)","(https, 48)","(presidente, 23)","(político, 20)","(ser, 15)","(político, 21)","(hacer, 127)"
4,"(implementar, 28)","(política, 17)","(lasso, 37)","(español, 21)","(politico, 18)","(político, 14)","(perú, 20)","(méxico, 117)"
5,"(https, 28)","(socialista, 17)","(política, 22)","(comunista, 20)","(decir, 17)","(política, 12)","(peruano, 15)","(españa, 114)"
6,"(federal, 28)","(gran, 17)","(lassoguillermo, 20)","(https, 18)","(https, 17)","(parte, 11)","(poder, 15)","(actual, 75)"
7,"(actual, 27)","(excluido, 16)","(hacer, 18)","(política, 17)","(argentina, 16)","(derecho, 10)","(hacer, 14)","(poder, 72)"
8,"(quédate, 27)","(próximo, 16)","(presidente, 14)","(sánchez, 15)","(poner, 14)","(poder, 10)","(comunista, 13)","(económico, 70)"
9,"(año, 22)","(dictador, 16)","(correa, 13)","(poder, 11)","(querer, 13)","(hacer, 10)","(corrupción, 12)","(presidente, 69)"


### "Constitución"

In [28]:
comparative_concordance(country_df, terms[6], country_list)

,Mexico:174,Colombia:172,Ecuador:55,Spain:291,Argentina:33,Chile:101,Peru:178,Total:1004
0,"(política, 109)","(colombia, 93)","(art, 8)","(española, 68)","(argentina, 7)","(nuevo, 32)","(perú, 81)","(política, 288)"
1,"(político, 31)","(política, 72)","(ecuador, 8)","(español, 35)","(nacional, 7)","(política, 19)","(política, 73)","(político, 144)"
2,"(mexicano, 31)","(político, 56)","(lasso, 8)","(día, 21)","(decir, 4)","(chile, 17)","(político, 33)","(colombia, 93)"
3,"(mexicana, 25)","(artículo, 22)","(130, 7)","(españa, 20)","(libertad, 2)","(república, 11)","(artículo, 22)","(perú, 83)"
4,"(méxico, 23)","(colombiano, 15)","(deber, 7)","(1978, 17)","(día, 2)","(político, 10)","(cambio, 13)","(artículo, 77)"
5,"(promulgación, 16)","(1991, 12)","(ser, 6)","(https, 16)","(duda, 2)","(nueva, 9)","(nuevo, 13)","(española, 68)"
6,"(artículo, 15)","(https, 11)","(destituir, 6)","(artículo, 13)","(articulo, 2)","(chileno, 8)","(cambiar, 11)","(nuevo, 60)"
7,"(origen, 14)","(ley, 10)","(trabajo, 4)","(política, 10)","(https, 2)","(deber, 8)","(peruano, 10)","(https, 47)"
8,"(conocer, 12)","(art, 9)","(poner, 4)","(político, 10)","(lamentablemente, 1)","(necesario, 6)","(https, 9)","(español, 36)"
9,"(1917, 12)","(colombiana, 9)","(artículo, 4)","(cumplir, 9)","(leyes, 1)","(poder, 6)","(hacer, 8)","(mexicano, 31)"


### "Democracia"

In [29]:
comparative_concordance(country_df, terms[7], country_list)

,Mexico:159,Colombia:180,Ecuador:109,Spain:261,Argentina:142,Chile:211,Peru:154,Total:1216
0,"(mexicano, 19)","(colombia, 28)","(ecuador, 17)","(españa, 47)","(político, 14)","(chile, 18)","(peruano, 16)","(político, 91)"
1,"(méxico, 16)","(colombiano, 24)","(https, 14)","(español, 40)","(argentina, 12)","(chileno, 17)","(perú, 12)","(https, 70)"
2,"(político, 10)","(colombiana, 13)","(político, 12)","(político, 22)","(vuelta, 9)","(político, 16)","(defender, 10)","(españa, 47)"
3,"(mexicana, 8)","(político, 12)","(código, 7)","(https, 22)","(argentino, 7)","(cristiana, 13)","(https, 9)","(español, 40)"
4,"(política, 7)","(https, 9)","(ecuatoriano, 7)","(española, 11)","(vivir, 6)","(corrupto, 11)","(hacer, 6)","(política, 34)"
5,"(libertad, 7)","(hacer, 7)","(libertad, 5)","(poder, 7)","(retorno, 6)","(alemania, 11)","(querer, 6)","(libertad, 30)"
6,"(participativo, 6)","(paz, 5)","(cristiana, 5)","(política, 7)","(https, 5)","(juzgar, 11)","(respetar, 6)","(colombia, 29)"
7,"(https, 5)","(verdadero, 5)","(decir, 4)","(vivir, 6)","(representación, 5)","(año, 9)","(poder, 6)","(poder, 25)"
8,"(poder, 5)","(creer, 5)","(existir, 3)","(separación, 6)","(país, 4)","(chilena, 7)","(peruana, 5)","(defender, 25)"
9,"(defensa, 4)","(país, 5)","(política, 3)","(libertad, 6)","(juarismo, 4)","(defender, 6)","(político, 5)","(hacer, 25)"


### "Libertad"

In [30]:
comparative_concordance(country_df, terms[8], country_list)

,Mexico:55,Colombia:51,Ecuador:49,Spain:130,Argentina:161,Chile:103,Peru:53,Total:602
0,"(expresión, 12)","(expresión, 10)","(privado, 9)","(expresión, 19)","(avanza, 26)","(chile, 19)","(expresión, 6)","(expresión, 65)"
1,"(méxico, 5)","(colombia, 8)","(expresión, 8)","(político, 17)","(argentina, 19)","(político, 13)","(prenso, 5)","(político, 49)"
2,"(libertad, 4)","(derecho, 5)","(ecuador, 7)","(españa, 16)","(acabar, 13)","(libertad, 10)","(prensa, 5)","(avanza, 26)"
3,"(político, 4)","(político, 3)","(sentenciar, 5)","(política, 8)","(dejar él, 13)","(mercado, 10)","(creer, 4)","(https, 24)"
4,"(democracia, 3)","(inclusión, 3)","(cárcel, 5)","(colectivo, 7)","(milagro, 10)","(expresión, 8)","(https, 4)","(libertad, 22)"
5,"(paladín, 3)","(colombiano, 2)","(https, 4)","(defender, 5)","(sala, 10)","(poder, 5)","(democracio, 3)","(argentina, 21)"
6,"(año, 3)","(privado, 2)","(democracia, 3)","(comunismo, 5)","(exigir, 9)","(patria, 4)","(querer, 3)","(justicia, 20)"
7,"(justicia, 3)","(libertad, 2)","(derecho, 3)","(justicia, 4)","(https, 9)","(paz, 4)","(castillo, 3)","(chile, 19)"
8,"(democracio, 3)","(democracia, 2)","(ecuatoriano, 3)","(viva, 4)","(idea, 8)","(comunismo, 4)","(sino, 3)","(democracia, 17)"
9,"(etc, 3)","(garantizar, 2)","(jorgeglas, 3)","(derecho, 4)","(carajo, 8)","(seguir, 4)","(aborto, 2)","(privado, 16)"


### "Partido"

In [31]:
comparative_concordance(country_df, terms[9], country_list)

,Mexico:311,Colombia:306,Ecuador:92,Spain:438,Argentina:249,Chile:386,Peru:345,Total:2127
0,"(político, 43)","(político, 54)","(político, 19)","(político, 46)","(futbol, 15)","(comunista, 74)","(perú, 40)","(político, 243)"
1,"(partido, 33)","(liberal, 26)","(ecuatoriano, 11)","(españa, 40)","(político, 13)","(político, 35)","(político, 34)","(comunista, 99)"
2,"(morena, 20)","(partido, 24)","(política, 10)","(popular, 30)","(ver, 13)","(chile, 29)","(morado, 31)","(partido, 96)"
3,"(solo, 18)","(colombia, 20)","(ecuador, 7)","(español, 24)","(poder, 9)","(republicano, 27)","(libre, 24)","(ser, 69)"
4,"(mexicano, 15)","(conservador, 18)","(rme, 6)","(ser, 21)","(hacer, 8)","(gente, 20)","(poder, 13)","(poder, 62)"
5,"(méxico, 13)","(nuevo, 16)","(organización, 6)","(vox, 16)","(mujer, 8)","(partido, 17)","(ser, 12)","(hacer, 59)"
6,"(poder, 13)","(verde, 15)","(https, 5)","(poder, 15)","(solo, 8)","(ser, 15)","(política, 11)","(política, 53)"
7,"(verde, 13)","(colombiano, 12)","(fuerza, 5)","(ninguno, 12)","(primero, 8)","(chileno, 15)","(peru, 11)","(solo, 52)"
8,"(cambiar, 11)","(hacer, 11)","(politico, 5)","(política, 11)","(votar, 8)","(corrupto, 11)","(desaparecer, 10)","(querer, 44)"
9,"(hacer, 11)","(querer, 10)","(social, 5)","(único, 11)","(argentina, 7)","(decir, 10)","(deber, 10)","(liberal, 44)"


### "Presidente"

In [32]:
comparative_concordance(country_df, terms[10], country_list)

,Mexico:567,Colombia:264,Ecuador:482,Spain:135,Argentina:302,Chile:317,Peru:501,Total:2568
0,"(méxico, 103)","(colombia, 64)","(lasso, 105)","(españa, 27)","(argentina, 25)","(chile, 85)","(castillo, 121)","(ser, 186)"
1,"(lopezobrador, 50)","(ser, 24)","(ecuador, 87)","(gobierno, 24)","(alberto, 22)","(ser, 50)","(perú, 66)","(castillo, 125)"
2,"(_, 50)","(petro, 21)","(lassoguillermo, 65)","(ser, 20)","(macri, 20)","(boric, 22)","(pedro, 61)","(político, 117)"
3,"(lópez, 39)","(https, 17)","(guillermo, 57)","(sánchez, 9)","(fernández, 18)","(kast, 21)","(peru, 28)","(méxico, 105)"
4,"(obrador, 38)","(hacer, 14)","(político, 38)","(peor, 7)","(milei, 17)","(poder, 18)","(ser, 27)","(lasso, 105)"
5,"(político, 31)","(ivanduque, 14)","(ser, 32)","(política, 6)","(ser, 16)","(futuro, 17)","(peruano, 24)","(https, 96)"
6,"(amlo, 26)","(duque, 12)","(mejor, 22)","(español, 6)","(decir, 14)","(próximo, 16)","(político, 23)","(chile, 89)"
7,"(andrés, 26)","(uribe, 12)","(https, 20)","(partido, 6)","(poder, 13)","(salir, 15)","(pedrocastillote, 23)","(ecuador, 89)"
8,"(manuel, 26)","(poner, 10)","(política, 16)","(sanchezcastejon, 5)","(https, 13)","(piñera, 15)","(destitución, 22)","(poder, 87)"
9,"(nel, 22)","(política, 9)","(económico, 16)","(pedro, 5)","(mejor, 9)","(chileno, 14)","(https, 20)","(hacer, 68)"


In [33]:
ibm_db.close(conn)

True